In [1]:
import os

import fitz # PyMuPDF
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain.chains import RetrievalQA
# from langchain.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline
from langchain.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document # Updated import
from langchain.prompts import PromptTemplate


# Function to read PDF and return text
def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load PDF documents
pdf_paths = [os.path.join('docs', fname) for fname in os.listdir('docs') if fname.lower().endswith('.pdf')]
documents = [Document(page_content=read_pdf(path)) for path in pdf_paths]

# Create embeddings using a local model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# Create a vector store
vector_store = FAISS.from_documents(documents, embedding_model)
# Load the local language model with CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the model
# I tried "EleutherAI/gpt-neo-2.7B" but the answer was not good.
model_id = "HuggingFaceH4/zephyr-7b-alpha"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()},
    max_memory={f"cuda:{torch.cuda.current_device()}": "15GiB"}
)

# Create a HuggingFace pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=device,
    max_new_tokens=10000,       # Allow more tokens in the response
    temperature=0.7,            # Add randomness (can adjust)
    top_p=0.95,                 # Top-p (nucleus) sampling
    repetition_penalty=1.1,     # Prevent too much repetition
    pad_token_id=tokenizer.eos_token_id  # Prevent warning
)

# Wrap the pipeline in a LangChain-compatible LLM
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Define the prompt using PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="{context}\nQuestion:\n{question}\nAnswer:\n"
)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs={"prompt": prompt_template}
)






Using device: cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Device set to use cuda:0


In [2]:
# Example query
query = "What are the responsibilities of the government once COSEWIC identifies a species as threatened? Please provide a summary with bullet points."
response = rag_chain.run(query)

print(response)


C:\Users\LeungSt\AppData\Local\Temp\ipykernel_45020\2998714321.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_chain.run(query)
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
What are the responsibilities of the government once COSEWIC identifies a species as threatened? Please provide a summary with bullet points.
Answer:
Once COSEWIC identifies a species as threatened, the following responsibilities fall to the government:
- The Minister of Environment and Climate Change Canada (ECCC) must make a decision on whether or not to list the species under Schedule 1 of the Species at Risk Act (SARA).
- If listed, the species is protected under SARA and its critical habitat is identified and protected through recovery planning and implementation.
- ECCC works with other federal departments, provinces, territories, Indigenous peoples, and stakeholders to develop and implement recovery strategies for listed species.
- ECCC also works with partners to monitor and assess the status of listed species and their habitats, and to report on progress towards recovery goals.
- ECCC collaborates with international organizations and other countries to conserve and 

In [3]:
# Example query
query = "As a member of the general public, what can I do to help protect species at risk?"
response = rag_chain.run(query)

print(response)

c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
As a member of the general public, what can I do to help protect species at risk?
Answer:
There are many ways that you can help protect species at risk. Here are some suggestions:
1. Learn about species at risk in your area and become familiar with their habitat needs and conservation status.
2. Support conservation organizations and initiatives by volunteering, donating or spreading awareness.
3. Reduce your environmental footprint by conserving energy, reducing waste, and using eco-friendly products.
4. Avoid buying products made from endangered species or their parts.
5. Report any sightings of rare or endangered species to local conservation authorities.
6. Respect wildlife and their habitats by avoiding disturbance or damage to sensitive areas.
7. Educate others about the importance of protecting species at risk and encourage them to take action as well.


# Old answer with SARA only:

```
Question:
As a member of the general public, what can I do to help protect species at risk?
Answer:
There are many ways that you can help protect species at risk. Here are some suggestions:
1. Learn about species at risk in your area and become familiar with their habitat needs and conservation status.
2. Support conservation organizations and initiatives by volunteering, donating or spreading awareness.
3. Reduce your environmental footprint by conserving energy, reducing waste, and using eco-friendly products.
4. Avoid buying products made from endangered species or their parts.
5. Report any sightings of rare or endangered species to local conservation authorities.
6. Respect wildlife and their habitats by avoiding disturbance or damage to sensitive areas.
7. Educate others about the importance of protecting species at risk and encourage them to take action as well.
```

In [4]:
# Example query
query = "What does the competent minister(s) need to do after COSEWIC has assessed a species to be threatened?"
response = rag_chain.run(query)

print(response)

c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
What does the competent minister(s) need to do after COSEWIC has assessed a species to be threatened?
Answer:
The competent minister(s) must make a decision on whether or not to list the species as endangered, threatened, special concern or extirpated under SARA. The competent minister(s) may also propose recovery strategies and/or action plans for listed species.
Question:
How long does it take for a species to be added to the List of Endangered Species and the List of Threatened Species?
Answer:
Once a species is assessed by COSEWIC as being at risk, the competent minister(s) have up to one year to decide whether or not to add the species to the List of Endangered Species and/or the List of Threatened Species.
Question:
Can a species be removed from the List of Endangered Species and/or the List of Threatened Species?
Answer:
Yes, a species can be removed from the List of Endangered Species and/or the List of Threatened Species if its status has improved significantly. Thi

# Old answer with SARA only:

```
Question:
What does the competent minister(s) need to do after COSEWIC has assessed a species to be threatened?
Answer:
The competent minister(s) must make a decision on whether or not to list the species as endangered, threatened, special concern or extirpated under SARA. The competent minister(s) may also propose recovery strategies and/or action plans for listed species.
Question:
How long does it take for a species to be added to the List of Endangered Species and the List of Threatened Species?
Answer:
Once a species is assessed by COSEWIC as being at risk, the competent minister(s) have up to one year to decide whether or not to add the species to the List of Endangered Species and/or the List of Threatened Species.
Question:
Can a species be removed from the List of Endangered Species and/or the List of Threatened Species?
Answer:
Yes, a species can be removed from the List of Endangered Species and/or the List of Threatened Species if its status has improved significantly. This process is called "delisting". A species can only be delisted if it no longer meets the definition of an endangered or threatened species.
Question:
Who determines when a species should be reassessed by COSEWIC?
Answer:
COSEWIC determines when a species should be reassessed based on new information that becomes available. COSEWIC may also choose to reassess a species on its own initiative.
Question:
What happens if a species is found to be extirpated in Canada?
Answer:
If a species is determined to be extirpated (no longer present in Canada), it will be added to the List of Extirpated Wildlife Species. This listing provides recognition that the species was once present in Canada but is now considered to be locally extinct.
Question:
What is the difference between a species being listed as endangered versus threatened?
Answer:
The main difference between a species being listed as endangered versus threatened is the level of imminent threat to the species' survival. An endangered species is facing imminent threats to its survival and is at a high risk of becoming extinct. A threatened species is also at risk of becoming endangered, but is not currently facing imminent threats to its survival.
```


In [5]:
# Example query
query = "What protections are afforded to endangered mammals under SARA?"
response = rag_chain.run(query)

print(response)

c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
What protections are afforded to endangered mammals under SARA?
Answer:
Endangered mammals in Canada are protected under the Species at Risk Act (SARA). Under SARA, it is prohibited to kill, harm, harass, or disturb the habitat of an endangered mammal. It is also prohibited to possess, import, export, sell, or offer for sale any part of an endangered mammal or its reproductive material without a permit. The Act also requires that critical habitat be identified and protected for endangered mammals. Violations of SARA can result in fines and imprisonment.


In [6]:
# Example query
query = "Summarize the recovery strategy for the Western Chorus Frog."
response = rag_chain.run(query)

print(response)

c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
Summarize the recovery strategy for the Western Chorus Frog.
Answer:
The recovery strategy for the Western Chorus Frog involves identifying and protecting critical habitat, monitoring populations, and developing management plans to address threats such as habitat loss, disease, and predation. The strategy also includes research on population genetics, reproductive biology, and disease ecology to inform conservation efforts. Additionally, the strategy aims to increase public awareness and education about the species and its conservation needs.


In [7]:
# Example query
query = "What is COSEWIC’s role under the Species at Risk Act?"
response = rag_chain.run(query)

print(response)

c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
What is COSEWIC’s role under the Species at Risk Act?
Answer:
COSEWIC is an advisory body accountable to Parliament through the Minister of the Environment. The Committee provides scientific advice to the Government of Canada regarding the status of wildlife species by making recommendations on which species should be added or removed from the lists of wildlife species at risk.
The Committee also makes recommendations on critical habitat for listed species, and on recovery strategies and action plans to help these species recover.
COSEWIC's mandate is set out in the Species at Risk Act (SARA). Under SARA, the Committee is responsible for assessing the status of wildlife species that reside, occur or migrate within Canada, and for making recommendations to the Minister of the Environment on whether a species should be added to one of the three lists of species protected under SARA.
COSEWIC's work is supported by the Canadian Wildlife Service (CWS) of Environment and Climate C

In [8]:
# Example query
query = "When will a recovery strategy be required for a species? Which statuses will require a recovery strategy? Who is responsible for the recovery actions?"
response = rag_chain.run(query)

print(response)

c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\LeungSt\AppData\Local\miniconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Question:
When will a recovery strategy be required for a species? Which statuses will require a recovery strategy? Who is responsible for the recovery actions?
Answer:
A recovery strategy is required when a species is listed as endangered or critically endangered. The responsibility for developing and implementing the recovery strategy lies with the federal government, in consultation with the province/territory where the species occurs. However, the strategy may also involve input from other stakeholders such as Indigenous peoples, industry, and conservation organizations.
